In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [39]:
app_df = pd.read_csv("../Downloads/application_record (1).csv")

In [40]:
app_df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


#  Credit dataset

In [42]:
credit = pd.read_csv("../Downloads/credit_record.csv")

In [43]:
grouped = credit.groupby('ID')

In [44]:
grouped

# Creating a pivot table to get a better overview.
### MONTHS_BALANCE: represents the financial snapshot of an account at a specific moment during a given month. 
### STATUS: To check whether a customer is paying his/her debts on time.

In [46]:
pivot_tb = credit.pivot(index = 'ID', columns = 'MONTHS_BALANCE', values = 'STATUS')

In [47]:
pivot_tb['open_month'] = grouped['MONTHS_BALANCE'].min()

In [48]:
pivot_tb['end_month'] = grouped['MONTHS_BALANCE'].max()

In [49]:
pivot_tb['ID'] = pivot_tb.index

In [50]:
pivot_tb = pivot_tb[['ID', 'open_month', 'end_month']]

In [51]:
pivot_tb['window'] = pivot_tb['end_month'] - pivot_tb['open_month']

In [52]:
pivot_tb.reset_index(drop = True, inplace = True)

# Merging tables:  credit  and pivot. 

In [54]:
credit = pd.merge(credit, pivot_tb, on = 'ID', how = 'left')

In [55]:
credit

,ID,MONTHS_BALANCE,STATUS,open_month,end_month,window
0,5001711,0,X,-3,0,3
1,5001711,-1,0,-3,0,3
2,5001711,-2,0,-3,0,3
3,5001711,-3,0,-3,0,3
4,5001712,0,C,-18,0,18
...,...,...,...,...,...,...
1048570,5150487,-25,C,-29,0,29
1048571,5150487,-26,C,-29,0,29
1048572,5150487,-27,C,-29,0,29
1048573,5150487,-28,C,-29,0,29


## Window: how many months it'll take a customer to reach their respective end_month.

In [57]:
pivot_tb['window'] = pivot_tb['end_month'] - pivot_tb['open_month']


In [58]:
pivot_tb.reset_index(drop = True, inplace = True)

In [59]:
pivot_tb

MONTHS_BALANCE,ID,open_month,end_month,window
0,5001711,-3,0,3
1,5001712,-18,0,18
2,5001713,-21,0,21
3,5001714,-14,0,14
4,5001715,-59,0,59
...,...,...,...,...
45980,5150482,-28,-11,17
45981,5150483,-17,0,17
45982,5150484,-12,0,12
45983,5150485,-1,0,1


# Customers whose window > 20 will be considered.
### i.e, it will take them more than 20 months to reach their respective end_month.

In [61]:
credit = credit[credit['window'] > 20]

In [62]:
credit

,ID,MONTHS_BALANCE,STATUS,open_month,end_month,window
23,5001713,0,X,-21,0,21
24,5001713,-1,X,-21,0,21
25,5001713,-2,X,-21,0,21
26,5001713,-3,X,-21,0,21
27,5001713,-4,X,-21,0,21
...,...,...,...,...,...,...
1048570,5150487,-25,C,-29,0,29
1048571,5150487,-26,C,-29,0,29
1048572,5150487,-27,C,-29,0,29
1048573,5150487,-28,C,-29,0,29


### Assigning labels: based on whether the customers are making their payments on time or not.
###  i.e; whose debts where late by more than 1 month are labeled as 1(bad), and rest as 0(good).

In [64]:
credit['status'] = np.where((credit['STATUS'] == '2') |
                            (credit['STATUS'] == '3' )|
                            (credit['STATUS'] == '4' )|
                            (credit['STATUS'] == '5'), 1, 0)

/var/folders/xh/dlr59jp10wj5hx57q_rhfy3c0000gn/T/ipykernel_69042/1704556549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit['status'] = np.where((credit['STATUS'] == '2') |


In [65]:
credit

,ID,MONTHS_BALANCE,STATUS,open_month,end_month,window,status
23,5001713,0,X,-21,0,21,0
24,5001713,-1,X,-21,0,21,0
25,5001713,-2,X,-21,0,21,0
26,5001713,-3,X,-21,0,21,0
27,5001713,-4,X,-21,0,21,0
...,...,...,...,...,...,...,...
1048570,5150487,-25,C,-29,0,29,0
1048571,5150487,-26,C,-29,0,29,0
1048572,5150487,-27,C,-29,0,29,0
1048573,5150487,-28,C,-29,0,29,0


In [66]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 775282 entries, 23 to 1048574
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              775282 non-null  int64 
 1   MONTHS_BALANCE  775282 non-null  int64 
 2   STATUS          775282 non-null  object
 3   open_month      775282 non-null  int64 
 4   end_month       775282 non-null  int64 
 5   window          775282 non-null  int64 
 6   status          775282 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 47.3+ MB


In [67]:
credit['status'] = credit['status'].astype(np.int8)

/var/folders/xh/dlr59jp10wj5hx57q_rhfy3c0000gn/T/ipykernel_69042/4214197907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit['status'] = credit['status'].astype(np.int8)


In [68]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 775282 entries, 23 to 1048574
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              775282 non-null  int64 
 1   MONTHS_BALANCE  775282 non-null  int64 
 2   STATUS          775282 non-null  object
 3   open_month      775282 non-null  int64 
 4   end_month       775282 non-null  int64 
 5   window          775282 non-null  int64 
 6   status          775282 non-null  int8  
dtypes: int64(5), int8(1), object(1)
memory usage: 42.1+ MB


In [69]:
credit.drop('STATUS', axis=1, inplace=True)

/var/folders/xh/dlr59jp10wj5hx57q_rhfy3c0000gn/T/ipykernel_69042/3339705583.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit.drop('STATUS', axis=1, inplace=True)


In [70]:
credit

,ID,MONTHS_BALANCE,open_month,end_month,window,status
23,5001713,0,-21,0,21,0
24,5001713,-1,-21,0,21,0
25,5001713,-2,-21,0,21,0
26,5001713,-3,-21,0,21,0
27,5001713,-4,-21,0,21,0
...,...,...,...,...,...,...
1048570,5150487,-25,-29,0,29,0
1048571,5150487,-26,-29,0,29,0
1048572,5150487,-27,-29,0,29,0
1048573,5150487,-28,-29,0,29,0


In [71]:
app_df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [72]:
app_df.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

In [73]:
app_df.dropna(inplace=True)

In [74]:
app_df.isna().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
dtype: int64

In [75]:
app_df.duplicated().sum()

0

In [76]:
app_df.drop(columns=['FLAG_OWN_CAR',
                    'FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE',
                    'DAYS_BIRTH','DAYS_EMPLOYED','FLAG_MOBIL','FLAG_WORK_PHONE',
                    'FLAG_PHONE','FLAG_EMAIL','CNT_FAM_MEMBERS'], inplace=True)

In [77]:
app_df

,ID,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE
2,5008806,M,0,112500.0,Married,House / apartment,Security staff
3,5008808,F,0,270000.0,Single / not married,House / apartment,Sales staff
4,5008809,F,0,270000.0,Single / not married,House / apartment,Sales staff
5,5008810,F,0,270000.0,Single / not married,House / apartment,Sales staff
6,5008811,F,0,270000.0,Single / not married,House / apartment,Sales staff
...,...,...,...,...,...,...,...
438541,6837707,M,0,202500.0,Civil marriage,House / apartment,Laborers
438548,6839936,M,1,135000.0,Married,House / apartment,Laborers
438553,6840222,F,0,103500.0,Single / not married,House / apartment,Laborers
438554,6841878,F,0,54000.0,Single / not married,With parents,Sales staff


In [78]:
application = pd.get_dummies(app_df, columns=['CODE_GENDER', 'OCCUPATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE']).astype(int)

In [79]:
application

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,CODE_GENDER_F,CODE_GENDER_M,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,...,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents
2,5008806,0,112500,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,5008808,0,270000,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,5008809,0,270000,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
5,5008810,0,270000,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
6,5008811,0,270000,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438541,6837707,0,202500,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
438548,6839936,1,135000,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
438553,6840222,0,103500,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
438554,6841878,0,54000,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [80]:
df = pd.merge(application,credit, on='ID', how='left')

In [81]:
df

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,CODE_GENDER_F,CODE_GENDER_M,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,...,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,MONTHS_BALANCE,open_month,end_month,window,status
0,5008806,0,112500,0,1,0,0,0,0,0,...,1,0,0,0,0,0.0,-29.0,0.0,29.0,0.0
1,5008806,0,112500,0,1,0,0,0,0,0,...,1,0,0,0,0,-1.0,-29.0,0.0,29.0,0.0
2,5008806,0,112500,0,1,0,0,0,0,0,...,1,0,0,0,0,-2.0,-29.0,0.0,29.0,0.0
3,5008806,0,112500,0,1,0,0,0,0,0,...,1,0,0,0,0,-3.0,-29.0,0.0,29.0,0.0
4,5008806,0,112500,0,1,0,0,0,0,0,...,1,0,0,0,0,-4.0,-29.0,0.0,29.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673632,6837707,0,202500,0,1,0,0,0,0,0,...,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN
673633,6839936,1,135000,0,1,0,0,0,0,0,...,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN
673634,6840222,0,103500,1,0,0,0,0,0,0,...,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN
673635,6841878,0,54000,1,0,0,0,0,0,0,...,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN


In [82]:
df.drop(columns=['MONTHS_BALANCE','open_month','end_month','window'], inplace=True)

In [83]:
df

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,CODE_GENDER_F,CODE_GENDER_M,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,...,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,status
0,5008806,0,112500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
1,5008806,0,112500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
2,5008806,0,112500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
3,5008806,0,112500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
4,5008806,0,112500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673632,6837707,0,202500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
673633,6839936,1,135000,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
673634,6840222,0,103500,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,NaN
673635,6841878,0,54000,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,NaN


In [84]:
df.dropna(inplace=True)

In [85]:
df.drop_duplicates(inplace=True)

In [86]:
df

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,CODE_GENDER_F,CODE_GENDER_M,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,...,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,status
0,5008806,0,112500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
32,5008810,0,270000,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0.0
59,5008811,0,270000,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0.0
99,5112956,0,270000,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
149,5008825,0,130500,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670939,5149158,0,247500,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1.0
670950,5149834,0,157500,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
670955,5149834,0,157500,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1.0
670974,5149838,0,157500,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0


In [87]:
df.drop(columns=['ID'], inplace=True)

In [88]:
df

,CNT_CHILDREN,AMT_INCOME_TOTAL,CODE_GENDER_F,CODE_GENDER_M,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,...,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,status
0,0,112500,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
32,0,270000,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0.0
59,0,270000,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0.0
99,0,270000,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
149,0,130500,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670939,0,247500,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1.0
670950,0,157500,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
670955,0,157500,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1.0
670974,0,157500,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0


#  Train-test-split

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
X = df.iloc[:,:-1]

In [92]:
y = df.iloc[:,-1]

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Upsampling the training set using SMOTE

In [95]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

## Since the dataset is imbalanced: 
### using Stratified cross-validation and fitting on Random forest classifier

In [97]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [98]:
cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = RandomForestClassifier()
scores = cross_val_score(model, X_resampled, y_resampled, cv=cv, scoring='accuracy')

In [99]:
mean_scores = np.mean(scores)

In [100]:
mean_scores

0.9450320705525671

In [101]:
model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [105]:
predictions = model.predict(X_test)

In [109]:
from sklearn.metrics import classification_report

In [111]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.97      0.92      0.95      2098
         1.0       0.08      0.19      0.11        67

    accuracy                           0.90      2165
   macro avg       0.52      0.56      0.53      2165
weighted avg       0.95      0.90      0.92      2165

